# **Forecasting with machine learning**

## Objectives

* use and ml model to forecast apple stock market data

## Accessing and visualising data

In [12]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error

In [13]:
# Some functions to help out with plotting
def plot_predictions(test,predicted):
    plt.plot(test, color='red',label='Real Apple Stock Price')
    plt.plot(predicted, color='blue',label='Predicted Apple Stock Price')
    plt.title('Apple Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Apple Stock Price')
    plt.legend()
    plt.show()

def return_rmse(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("The root mean squared error is {}.".format(rmse))

In [14]:
from pathlib import Path
import pandas as pd

path = Path('../stockdata/AAPL_2006-01-01_to_2018-01-01.csv')

apple_stock = pd.read_csv(
    path,
    usecols=['Date','Close'],
    parse_dates =['Date'],
    index_col='Date',
) 

y = apple_stock.loc[:, 'Close'].dropna().to_period('Date')

df = pd.DataFrame

ValueError: Invalid frequency: Date

In [8]:
training_set = apple_stock[:'2016'].iloc[:,1:2].values
test_set = apple_stock['2017':].iloc[:,1:2].values

In [9]:
print(training_set)

[]


In [10]:
# Plot of Close attribute for stock prices for train and test sets
apple_stock["Close"][:'2016'].plot(figsize=(16,4),legend=True)
apple_stock["Close"]['2017':].plot(figsize=(16,4),legend=True)
plt.legend(['Training set (Before 2017)','Test set (2017 and beyond)'])
plt.title('Apple stock price')
plt.show()

KeyError: 'Close'

In [ ]:
# Scaling the training set
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

X_train = []
y_train = []
for i in range(60,2768):
    X_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
# Reshape for efficitent modelling
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

## Modelling

In [ ]:
# The LSTM architecture
regressor = Sequential()
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units=1))

# Compiling the RNN
regressor.compile(optimizer='rmsprop',loss='mean_squared_error')
# Summary of the model
regressor.summary()

2023-01-31 22:09:10.326943: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-31 22:09:10.326986: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-31 22:09:10.327010: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kylo74-timeseriesanalys-m4dh5repsh7): /proc/driver/nvidia/version does not exist
2023-01-31 22:09:10.327267: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 50)            10400     
                                                                 
 dropout (Dropout)           (None, 60, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_1 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_2 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_2 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_3 (LSTM)               (None, 50)                2

### Fitting the model to the training data set

In [ ]:
regressor.fit(X_train,y_train,epochs=50,batch_size=32)

Epoch 1/50
85/85 [==============================] - 14s 93ms/step - loss: 0.0165
Epoch 2/50
85/85 [==============================] - 6s 72ms/step - loss: 0.0067
Epoch 3/50
85/85 [==============================] - 6s 69ms/step - loss: 0.0060
Epoch 4/50
85/85 [==============================] - 6s 68ms/step - loss: 0.0050
Epoch 5/50
85/85 [==============================] - 6s 70ms/step - loss: 0.0045
Epoch 6/50
85/85 [==============================] - 6s 77ms/step - loss: 0.0044
Epoch 7/50
85/85 [==============================] - 6s 76ms/step - loss: 0.0037
Epoch 8/50
85/85 [==============================] - 7s 79ms/step - loss: 0.0035
Epoch 9/50
85/85 [==============================] - 6s 75ms/step - loss: 0.0032
Epoch 10/50
85/85 [==============================] - 7s 83ms/step - loss: 0.0031
Epoch 11/50
85/85 [==============================] - 7s 83ms/step - loss: 0.0029
Epoch 12/50
85/85 [==============================] - 6s 76ms/step - loss: 0.0027
Epoch 13/50
85/85 [=================

In [ ]:
apple_stock_total = pd.concat((apple_stock["Close"][:'2016'],apple_stock["Close"]['2017':]),axis=0)
inputs = apple_stock_total[len(apple_stock_total)-len(test_set) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = sc.transform(inputs)

## Predicting prices and visualising predictions

In [ ]:
X_test = []
for i in range(60,311):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

8/8 [==============================] - 1s 31ms/step


In [ ]:
plot_predictions(test_set,predicted_stock_price)

NameError: name 'plot_predictions' is not defined

## Model Evaluation


In [ ]:
# Evaluating our model
return_rmse(test_set,predicted_stock_price)

NameError: name 'return_rmse' is not defined